# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.
/opt/conda/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/learn-env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/learn-env/lib/python3.6/site-pa

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [23]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation = 'relu', input_shape = (n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation = 'relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation = 'linear'))



# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 185us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 63us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 86us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 74us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 69us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 64us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 71us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 61us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 70us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 70us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 74us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 55us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 73us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 62us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 54us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 54us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 53us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 69

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [20]:
type(numeric_columns)

list

In [45]:
numeric_columns

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

In [8]:
X_train[numeric_columns]

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,540.0,0.0,130.0,0.0,130.0,0.0,0.0,0.0,10.0,2006.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,622.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,6.0,2009.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,534.0,171.0,159.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,608.0,237.0,152.0,0.0,0.0,0.0,0.0,0.0,3.0,2010.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,812.0,168.0,52.0,0.0,0.0,0.0,0.0,0.0,4.0,2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,20.0,69.0,6627.0,3.0,6.0,1949.0,1950.0,0.0,0.0,0.0,...,287.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,2008.0
1047,60.0,77.0,9965.0,7.0,5.0,2007.0,2007.0,220.0,984.0,0.0,...,792.0,120.0,184.0,0.0,0.0,168.0,0.0,0.0,10.0,2007.0
1048,20.0,91.0,12274.0,7.0,5.0,2008.0,2008.0,256.0,0.0,0.0,...,554.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,7.0,2008.0
1049,20.0,69.0,4403.0,7.0,5.0,2009.0,2009.0,432.0,578.0,0.0,...,484.0,0.0,144.0,0.0,0.0,0.0,0.0,0.0,6.0,2010.0


In [26]:
X_train_cat

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,Alley_missing,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1047,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1048,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1049,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
X_train_scaled

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.501710,-0.023913,1.393427,-0.087591,-0.529609,-0.064317,-0.759794,-0.573810,1.049194,-0.272846,...,0.299616,-0.753090,1.147764,-0.376852,3.945189,-0.265709,-0.061029,-0.110065,1.364143,-1.350321
1,0.036311,0.426489,0.258820,0.643027,-0.529609,0.975634,0.782447,-0.573810,0.462395,-0.272846,...,0.687425,-0.753090,-0.067011,-0.376852,-0.119841,-0.265709,-0.061029,-0.110065,-0.128532,0.876437
2,-0.894487,0.246328,-0.077556,1.373645,-0.529609,1.008133,0.830642,1.013049,-0.974399,-0.272846,...,0.271240,0.575435,1.562216,-0.376852,-0.119841,-0.265709,-0.061029,-0.110065,0.990974,0.134185
3,1.432507,-1.375120,-0.624323,1.373645,-0.529609,0.780643,0.541472,-0.573810,1.571273,-0.272846,...,0.621214,1.088199,1.462176,-0.376852,-0.119841,-0.265709,-0.061029,-0.110065,-1.248038,1.618690
4,0.036311,0.696731,0.080834,2.834880,-0.529609,1.203124,1.119812,0.329740,0.675973,-0.272846,...,1.586008,0.552127,0.033029,-0.376852,-0.119841,-0.265709,-0.061029,-0.110065,-0.874869,0.876437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,-0.894487,-0.023913,-0.468822,-2.279444,0.384380,-0.714286,-1.675499,-0.573810,-0.974399,-0.272846,...,-0.896918,-0.753090,-0.710127,-0.376852,-0.119841,-0.265709,-0.061029,-0.110065,0.244637,0.134185
1047,0.036311,0.336409,-0.049546,0.643027,-0.529609,1.170625,1.071617,0.668571,1.148432,-0.272846,...,1.491420,0.179208,1.919503,-0.376852,-0.119841,2.781364,-0.061029,-0.110065,1.364143,-0.608068
1048,-0.894487,0.966972,0.240481,0.643027,-0.529609,1.203124,1.119812,0.871870,-0.974399,-0.272846,...,0.365827,-0.753090,0.147361,-0.376852,-0.119841,-0.265709,-0.061029,-0.110065,0.244637,0.134185
1049,-0.894487,-0.023913,-0.748171,0.643027,-0.529609,1.235622,1.168007,1.865774,0.272549,-0.272846,...,0.034771,-0.753090,1.347844,-0.376852,-0.119841,-0.265709,-0.061029,-0.110065,-0.128532,1.618690


In [16]:
# Numeric column names
numeric_columns = list(X_train_numeric.columns.values)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train[numeric_columns]), columns = numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val[numeric_columns]), columns = numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test[numeric_columns]), columns = numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [17]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [18]:
# Train the model 
normalized_input_model.fit(X_train, y_train, batch_size = 32, epochs = 150, validation_data = (X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 179us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 68us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 65us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 71us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 72us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 79us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [=====

1051/1051 [==============================] - 0s 77us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 76us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 68us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 71us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 80us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 75us/step - loss: nan 

Epoch 106/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 55us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 54us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 59us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 61us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 81us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 112/150
1051/1051 [==============================] - 0s 77

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [19]:
y_train

,SalePrice
0,180000
1,230000
2,230000
3,236500
4,310000
...,...
1046,72500
1047,383970
1048,227680
1049,222000


In [24]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = pd.DataFrame(ss_y.fit_transform(y_train))

# Transform validate and test labels
y_val_scaled = pd.DataFrame(ss_y.transform(y_val))
y_test_scaled = pd.DataFrame(ss_y.transform(y_test))

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [32]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, batch_size = 32, epochs = 150, validation_data = (X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 349us/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.2074 - val_mean_squared_error: 0.2074
Epoch 2/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.2274 - mean_squared_error: 0.2274 - val_loss: 0.1697 - val_mean_squared_error: 0.1697
Epoch 3/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1481 - val_mean_squared_error: 0.1481
Epoch 4/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.1643 - mean_squared_error: 0.1643 - val_loss: 0.1535 - val_mean_squared_error: 0.1535
Epoch 5/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.1481 - mean_squared_error: 0.1481 - val_loss: 0.1313 - val_mean_squared_error: 0.1313
Epoch 6/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0317 - mean_squared_error: 0.0317 - val_loss: 0.1025 - val_mean_squared_error: 0.1025
Epoch 51/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0316 - mean_squared_error: 0.0316 - val_loss: 0.1023 - val_mean_squared_error: 0.1023
Epoch 52/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0308 - mean_squared_error: 0.0308 - val_loss: 0.1005 - val_mean_squared_error: 0.1005
Epoch 53/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0301 - mean_squared_error: 0.0301 - val_loss: 0.1049 - val_mean_squared_error: 0.1049
Epoch 54/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0291 - mean_squared_error: 0.0291 - val_loss: 0.1081 - val_mean_squared_error: 0.1081
Epoch 55/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0285 - mean_squared_error: 0.0285 - val_loss: 0.1013 - val_mean_squared_error: 0.101

1051/1051 [==============================] - 0s 61us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1039 - val_mean_squared_error: 0.1039
Epoch 100/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1106 - val_mean_squared_error: 0.1106
Epoch 101/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1138 - val_mean_squared_error: 0.1138
Epoch 102/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1113 - val_mean_squared_error: 0.1113
Epoch 103/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0135 - mean_squared_error: 0.0135 - val_loss: 0.1076 - val_mean_squared_error: 0.1076
Epoch 104/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0132 - mean_squared_error: 0.0132 - val_loss: 0.1154 - val_mean_squared_error: 0.1154
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 78us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1140 - val_mean_squared_error: 0.1140
Epoch 149/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.1115 - val_mean_squared_error: 0.1115
Epoch 150/150
1051/1051 [==============================] - 0s 84us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1122 - val_mean_squared_error: 0.1122


In [27]:
X_train.head(3)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.501710,-0.023913,1.393427,-0.087591,-0.529609,-0.064317,-0.759794,-0.573810,1.049194,-0.272846,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.036311,0.426489,0.258820,0.643027,-0.529609,0.975634,0.782447,-0.573810,0.462395,-0.272846,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.894487,0.246328,-0.077556,1.373645,-0.529609,1.008133,0.830642,1.013049,-0.974399,-0.272846,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [29]:
y_train_scaled.head(3)

,0
0,-0.024946
1,0.611320
2,0.611320


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 119us/step


[0.007758111467888024, 0.007758111467888024]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [34]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 74us/step


[0.11222613744522682, 0.11222613744522682]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [35]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
mean_squared_error(y_val, y_val_pred)

693038000.2971045

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [37]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer = 'he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 263us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1872 - val_mean_squared_error: 0.1872
Epoch 2/150
1051/1051 [==============================] - 0s 91us/step - loss: 0.2217 - mean_squared_error: 0.2217 - val_loss: 0.1567 - val_mean_squared_error: 0.1567
Epoch 3/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.1798 - mean_squared_error: 0.1798 - val_loss: 0.1387 - val_mean_squared_error: 0.1387
Epoch 4/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.1568 - mean_squared_error: 0.1568 - val_loss: 0.1461 - val_mean_squared_error: 0.1461
Epoch 5/150
1051/1051 [==============================] - 0s 86us/step - loss: 0.1410 - mean_squared_error: 0.1410 - val_loss: 0.1319 - val_mean_squared_error: 0.1319
Epoch 6/150
1051/1051 [==============================] - 0s 96us/step - loss: 0.1265 - mean_squared_error: 0.1265 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 88us/step - loss: 0.0304 - mean_squared_error: 0.0304 - val_loss: 0.1026 - val_mean_squared_error: 0.1026
Epoch 51/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0297 - mean_squared_error: 0.0297 - val_loss: 0.1070 - val_mean_squared_error: 0.1070
Epoch 52/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0293 - mean_squared_error: 0.0293 - val_loss: 0.1060 - val_mean_squared_error: 0.1060
Epoch 53/150
1051/1051 [==============================] - 0s 82us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1082 - val_mean_squared_error: 0.1082
Epoch 54/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.1116 - val_mean_squared_error: 0.1116
Epoch 55/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0278 - mean_squared_error: 0.0278 - val_loss: 0.1094 - val_mean_squared_error: 0.109

1051/1051 [==============================] - 0s 70us/step - loss: 0.0145 - mean_squared_error: 0.0145 - val_loss: 0.1097 - val_mean_squared_error: 0.1097
Epoch 100/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1191 - val_mean_squared_error: 0.1191
Epoch 101/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1204 - val_mean_squared_error: 0.1204
Epoch 102/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 103/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1173 - val_mean_squared_error: 0.1173
Epoch 104/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1246 - val_mean_squared_error: 0.1246
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1235 - val_mean_squared_error: 0.1235
Epoch 149/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1201 - val_mean_squared_error: 0.1201
Epoch 150/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1177 - val_mean_squared_error: 0.1177


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [38]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 109us/step


[0.008204983050654067, 0.008204983050654067]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [39]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 52us/step


[0.11766200964498429, 0.11766200964498429]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [40]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer = 'lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 479us/step - loss: 0.4769 - mean_squared_error: 0.4769 - val_loss: 0.1646 - val_mean_squared_error: 0.1646
Epoch 2/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.2096 - mean_squared_error: 0.2096 - val_loss: 0.1371 - val_mean_squared_error: 0.1371
Epoch 3/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.1698 - mean_squared_error: 0.1698 - val_loss: 0.1225 - val_mean_squared_error: 0.1225
Epoch 4/150
1051/1051 [==============================] - 0s 86us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1291 - val_mean_squared_error: 0.1291
Epoch 5/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.1330 - mean_squared_error: 0.1330 - val_loss: 0.1165 - val_mean_squared_error: 0.1165
Epoch 6/150
1051/1051 [==============================] - 0s 83us/step - loss: 0.1196 - mean_squared_error: 0.1196 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0267 - mean_squared_error: 0.0267 - val_loss: 0.1036 - val_mean_squared_error: 0.1036
Epoch 51/150
1051/1051 [==============================] - 0s 60us/step - loss: 0.0262 - mean_squared_error: 0.0262 - val_loss: 0.1075 - val_mean_squared_error: 0.1075
Epoch 52/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0259 - mean_squared_error: 0.0259 - val_loss: 0.1061 - val_mean_squared_error: 0.1061
Epoch 53/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.1088 - val_mean_squared_error: 0.1088
Epoch 54/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0248 - mean_squared_error: 0.0248 - val_loss: 0.1110 - val_mean_squared_error: 0.1110
Epoch 55/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0244 - mean_squared_error: 0.0244 - val_loss: 0.1088 - val_mean_squared_error: 0.108

1051/1051 [==============================] - 0s 62us/step - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.1073 - val_mean_squared_error: 0.1073
Epoch 100/150
1051/1051 [==============================] - 0s 87us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1180 - val_mean_squared_error: 0.1180
Epoch 101/150
1051/1051 [==============================] - 0s 75us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1202 - val_mean_squared_error: 0.1202
Epoch 102/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0123 - mean_squared_error: 0.0123 - val_loss: 0.1168 - val_mean_squared_error: 0.1168
Epoch 103/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1168 - val_mean_squared_error: 0.1168
Epoch 104/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.1210 - val_mean_squared_error: 0.1210
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1224 - val_mean_squared_error: 0.1224
Epoch 149/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1173 - val_mean_squared_error: 0.1173
Epoch 150/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1146 - val_mean_squared_error: 0.1146


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [41]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 32us/step


[0.007061840136040119, 0.007061840136040119]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [42]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 145us/step


[0.11463621777160087, 0.11463621777160087]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [43]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                    loss='mse',
                    metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 442us/step - loss: 0.3205 - mean_squared_error: 0.3205 - val_loss: 0.1386 - val_mean_squared_error: 0.1386
Epoch 2/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.1667 - mean_squared_error: 0.1667 - val_loss: 0.1007 - val_mean_squared_error: 0.1007
Epoch 3/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.1243 - mean_squared_error: 0.1243 - val_loss: 0.1051 - val_mean_squared_error: 0.1051
Epoch 4/150
1051/1051 [==============================] - 0s 72us/step - loss: 0.0988 - mean_squared_error: 0.0988 - val_loss: 0.1238 - val_mean_squared_error: 0.1238
Epoch 5/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.0887 - mean_squared_error: 0.0887 - val_loss: 0.1218 - val_mean_squared_error: 0.1218
Epoch 6/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0655 - mean_squared_error: 0.0655 - val_lo

Epoch 50/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0088 - mean_squared_error: 0.0088 - val_loss: 0.0966 - val_mean_squared_error: 0.0966
Epoch 51/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.0076 - mean_squared_error: 0.0076 - val_loss: 0.0891 - val_mean_squared_error: 0.0891
Epoch 52/150
1051/1051 [==============================] - 0s 89us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1009 - val_mean_squared_error: 0.1009
Epoch 53/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0070 - mean_squared_error: 0.0070 - val_loss: 0.0938 - val_mean_squared_error: 0.0938
Epoch 54/150
1051/1051 [==============================] - 0s 77us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0930 - val_mean_squared_error: 0.0930
Epoch 55/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0064 - mean_squared_error: 0.0064 - val_loss: 0.0892 - val_mean_squared_error: 0.089

1051/1051 [==============================] - 0s 75us/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.1028 - val_mean_squared_error: 0.1028
Epoch 100/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.0853 - val_mean_squared_error: 0.0853
Epoch 101/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0929 - val_mean_squared_error: 0.0929
Epoch 102/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0049 - mean_squared_error: 0.0049 - val_loss: 0.0865 - val_mean_squared_error: 0.0865
Epoch 103/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.0050 - mean_squared_error: 0.0050 - val_loss: 0.0902 - val_mean_squared_error: 0.0902
Epoch 104/150
1051/1051 [==============================] - 0s 69us/step - loss: 0.0055 - mean_squared_error: 0.0055 - val_loss: 0.0872 - val_mean_squared_error: 0.0872
Epoch 

Epoch 148/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.0889 - val_mean_squared_error: 0.0889
Epoch 149/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0861 - val_mean_squared_error: 0.0861
Epoch 150/150
1051/1051 [==============================] - 0s 86us/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0870 - val_mean_squared_error: 0.0870


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [44]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 56us/step


[0.007291534341354239, 0.007291534341354239]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [45]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 368us/step


[0.08695570662787659, 0.08695570662787659]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [46]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                    loss='mse',
                    metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 516us/step - loss: 0.3728 - mean_squared_error: 0.3728 - val_loss: 0.1694 - val_mean_squared_error: 0.1694
Epoch 2/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.1566 - mean_squared_error: 0.1566 - val_loss: 0.1243 - val_mean_squared_error: 0.1243
Epoch 3/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.1098 - mean_squared_error: 0.1098 - val_loss: 0.1009 - val_mean_squared_error: 0.1009
Epoch 4/150
1051/1051 [==============================] - 0s 98us/step - loss: 0.0852 - mean_squared_error: 0.0852 - val_loss: 0.1158 - val_mean_squared_error: 0.1158
Epoch 5/150
1051/1051 [==============================] - 0s 95us/step - loss: 0.0717 - mean_squared_error: 0.0717 - val_loss: 0.0991 - val_mean_squared_error: 0.0991
Epoch 6/150
1051/1051 [==============================] - 0s 105us/step - loss: 0.0548 - mean_squared_error: 0.0548 - val_l

Epoch 50/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.1143 - val_mean_squared_error: 0.1143
Epoch 51/150
1051/1051 [==============================] - 0s 99us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.1149 - val_mean_squared_error: 0.1149
Epoch 52/150
1051/1051 [==============================] - 0s 81us/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.1087 - val_mean_squared_error: 0.1087
Epoch 53/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0027 - mean_squared_error: 0.0027 - val_loss: 0.1088 - val_mean_squared_error: 0.1088
Epoch 54/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.1089 - val_mean_squared_error: 0.1089
Epoch 55/150
1051/1051 [==============================] - 0s 70us/step - loss: 0.0033 - mean_squared_error: 0.0033 - val_loss: 0.1152 - val_mean_squared_error: 0.115

1051/1051 [==============================] - 0s 78us/step - loss: 0.0080 - mean_squared_error: 0.0080 - val_loss: 0.0915 - val_mean_squared_error: 0.0915
Epoch 100/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.1028 - val_mean_squared_error: 0.1028
Epoch 101/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0021 - mean_squared_error: 0.0021 - val_loss: 0.0965 - val_mean_squared_error: 0.0965
Epoch 102/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0999 - val_mean_squared_error: 0.0999
Epoch 103/150
1051/1051 [==============================] - 0s 81us/step - loss: 8.4083e-04 - mean_squared_error: 8.4083e-04 - val_loss: 0.0982 - val_mean_squared_error: 0.0982
Epoch 104/150
1051/1051 [==============================] - 0s 77us/step - loss: 7.5049e-04 - mean_squared_error: 7.5049e-04 - val_loss: 0.0989 - val_mean_squared_erro

1051/1051 [==============================] - 0s 90us/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0949 - val_mean_squared_error: 0.0949
Epoch 148/150
1051/1051 [==============================] - 0s 96us/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0953 - val_mean_squared_error: 0.0953
Epoch 149/150
1051/1051 [==============================] - 0s 90us/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0955 - val_mean_squared_error: 0.0955
Epoch 150/150
1051/1051 [==============================] - 0s 80us/step - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 0.0967 - val_mean_squared_error: 0.0967


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [47]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 71us/step


[0.0012924741165177254, 0.0012924741165177254]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [48]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 301us/step


[0.09670058644939739, 0.09670058644939739]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [49]:
# Evaluate the best model on test data
adam_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 669us/step


[0.17688818255515948, 0.17688818255515948]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [53]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
mean_squared_error(y_test, y_test_pred)

1092350016.939492

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.